In [ ]:
import os
import argparse

import torch
import cv2

from test import GOTURN


def axis_aligned_iou(boxA, boxB):
    # make sure that x1,y1,x2,y2 of a box are valid
    assert(boxA[0] <= boxA[2])
    assert(boxA[1] <= boxA[3])
    assert(boxB[0] <= boxB[2])
    assert(boxB[1] <= boxB[3])

    # determine the (x, y)-coordinates of the intersection rectangle
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])

    # compute the area of intersection rectangle
    interArea = max(0, xB - xA + 1) * max(0, yB - yA + 1)

    # compute the area of both the prediction and ground-truth
    # rectangles
    boxAArea = (boxA[2] - boxA[0] + 1) * (boxA[3] - boxA[1] + 1)
    boxBArea = (boxB[2] - boxB[0] + 1) * (boxB[3] - boxB[1] + 1)

    # compute the intersection over union by taking the intersection
    # area and dividing it by the sum of prediction + ground-truth
    # areas - the interesection area
    iou = interArea / float(boxAArea + boxBArea - interArea)

    # return the intersection over union value
    return iou


def save(im, bb, gt_bb, idx):
#    f= open('centroids_results.txt','a')
#    l = str(idx)+','+str(bb[0])+','+str(bb[1])+','+str(bb[2])+','+str(bb[3])+'\n'
#    f.writelines(l)
#    f.close()    
        
    im = cv2.cvtColor(im, cv2.COLOR_RGB2BGR)
    bb = [int(val) for val in bb]  # GOTURN output
    gt_bb = [int(val) for val in gt_bb]  # groundtruth box
    # plot GOTURN predictions with red rectangle
    im = cv2.rectangle(im, (bb[0], bb[1]), (bb[2], bb[3]),
                       (0, 0, 255), 2)
    # plot annotations with white rectangle
    im = cv2.rectangle(im, (gt_bb[0], gt_bb[1]), (gt_bb[2], gt_bb[3]),
                       (255, 255, 255), 2)
    save_path = os.path.join(args.save_directory, str(idx)+'.jpg')
    cv2.imwrite(save_path, im)

names = ['bear','camel','book','rhino','swan']
path_in = "./pygoturn-master/data/OTB"
path_out = "./result/"
model_weights = "./model/pytorch_goturn.pth.tar"

def main(args):
    for n in names :
        cuda = torch.cuda.is_available()
        device = torch.device('cuda:0' if cuda else 'cpu')
        tester = GOTURN(path_in+n,
                        model_weights,
                        device)

        if os.path.exists(path_out+n):
            print('Save directory %s already exists' % (path_out+n))
        else:
            os.makedirs(path_out+n)
        # save initial frame with bounding box
        save(tester.img[0][0], tester.prev_rect, tester.prev_rect, 1)
        tester.model.eval()

        # loop through sequence images
        for i in range(tester.len):
            # get torch input tensor
            sample = tester[i]

            # predict box
            bb = tester.get_rect(sample)
            gt_bb = tester.gt[i]
            tester.prev_rect = bb

            # save current image with predicted rectangle and gt box
            im = tester.img[i][1]
            save(im, bb, gt_bb, i+2)

            # print stats
            print('frame: %d, IoU = %f' % (
                i+2, axis_aligned_iou(gt_bb, bb)))


#if __name__ == "__main__":
   # args = parser.parse_args()
  #  main(args)